# Modelling the DEZEMBER Alternation

In this notebook, the DEZEMBER alternation is modelled as described in the doctoral thesis. Refer to the relevant chapter to understand modelling choices.

Note that in this notebook, the variable for the logarithmised distance is called PREVIOUS_DISTANCE_LOG, whereas it is simply called PREVIOUS_DISTANCE in the doctoral thesis. Despite lacking the specifier "_LOG" in the thesis, the variables in the formulae always refer to the logarithmised version of the distance measure.

## Preparations

### Importing Relevant Libraries

In [ ]:
library(lme4)
library(glue)
library(performance)
library(broom)
library(scales)
library(sjPlot)
library(ggplot2)
library(dplyr)
library(stargazer)
library(broom.mixed)

### Loading and Preprocessing Data

In [ ]:
data <- read.csv("DEZEMBER_for_modelling.csv")

Treatment coding of response variable: "dezember" = 0, "zwölf" = 1.

In [ ]:
data$CURRENT <- as.factor(data$CURRENT)
cat("R models this level:", tail(levels(data$CURRENT), 1))

Renaming speakers for consistency with the rest of the thesis.

In [ ]:
data <- data %>% mutate(PREVIOUS_SPEAKER = recode(PREVIOUS_SPEAKER, "A" = "VA", "S" = "HS"))

Combining the variables PREVIOUS and PREVIOUS_SPEAKER, as modelling an interaction of these two independent variables leads to an issue of singularity.

In [ ]:
data$PREVIOUS_SPEAKER_COMBINED <- interaction(data$PREVIOUS, data$PREVIOUS_SPEAKER) #R automatically creates a factor
cat("Levels are:", levels(data$PREVIOUS_SPEAKER_COMBINED))

`interaction()` also creates combinations which never appear. 

In [ ]:
#discarding empty levels
data$PREVIOUS_SPEAKER_COMBINED <- droplevels(data$PREVIOUS_SPEAKER_COMBINED)

#before renaming (see above) "dezember.A" was the reference level as it is first in alphabetical order, 
#to preserve this even after renaming, the factor is relevelled 
data$PREVIOUS_SPEAKER_COMBINED <- relevel(data$PREVIOUS_SPEAKER_COMBINED, ref = "dezember.VA")
cat("The reference level of PREVIOUS_SPEAKER_COMBINED is:", head(levels(data$PREVIOUS_SPEAKER_COMBINED), 1)) 

Treatment coding of further predictor variables.

In [ ]:
data <- data %>% rename(PREVIOUS_BETA_ZWOELF = PREVIOUS_BETA_ZWÖLF) #renaming to avoid Umlaut issue
data$PREVIOUS_BETA_ZWOELF <- as.factor(data$PREVIOUS_BETA_ZWOELF) 
data$QUASI_PERSISTENCE <- as.factor(data$QUASI_PERSISTENCE) 
data$CONFEDERATE <- as.factor(data$CONFEDERATE) 

## Simple Model

### Model Fitting

Fitting a simple regression model without any random effects to assess how the predictor variables impact the response variable.

In [ ]:
simple_model <- glm(CURRENT ~ PREVIOUS_SPEAKER_COMBINED * PREVIOUS_DISTANCE_LOG + 
                                     PREVIOUS_SPEAKER_COMBINED * QUASI_PERSISTENCE +
                                     PREVIOUS_BETA_ZWOELF,
                                     data = data, family = 'binomial')

summary(simple_model)

### Model Evaluation

#### R² and AIC

In [ ]:
r2_nagelkerke(simple_model)
AIC(simple_model)

#### Predictive Efficiency

In [ ]:
#Predicting based on simple_model
fixed_model_predictions <- predict(simple_model, newdata = data, type = "response")
#Convert probabilities to binary outcomes (i.e., if probability > 0.5, predict 1, else 0)
fixed_model_predicted_class <- ifelse(fixed_model_predictions > 0.5, "zwoelf", "dezember")
#Compare predicted values to actual values
fixed_model_accuracy <- mean(fixed_model_predicted_class == data$CURRENT)
fixed_model_accuracy

In [ ]:
#Calculate baseline accuracy, i.e., a dumb intercept-only model only ever predicting the most frequent outcome
counts <- table(data$CURRENT) 
dumb_model_accuracy <- max(counts) / sum(counts)
dumb_model_accuracy

In [ ]:
#McNemar's Test for significance against baseline
baseline_predicted_class <- rep(names(which.max(counts)), length(data$CURRENT))  #Create baseline predictions (always predict the most frequent outcome)
mcnemar_table <- table(
  model_correct = (fixed_model_predicted_class == data$CURRENT),
  baseline_correct = (baseline_predicted_class == data$CURRENT)) #Create a contingency table: Compare model and baseline predictions against actual values
mcnemar_result <- mcnemar.test(mcnemar_table) #Perform McNemar's Test
mcnemar_result

## Mixed Model

### Model Fitting

In [ ]:
initial_mixed_model <- glmer(CURRENT ~ PREVIOUS_SPEAKER_COMBINED * PREVIOUS_DISTANCE_LOG + 
                                      PREVIOUS_BETA_ZWOELF +
                                      (1 | HUMAN_ID), 
                                      data = data, family = binomial)

summary(initial_mixed_model)

In [ ]:
final_mixed_model <- glmer(CURRENT ~ PREVIOUS_SPEAKER_COMBINED * QUASI_PERSISTENCE +
                                         PREVIOUS_BETA_ZWOELF + 
                                         (1 | HUMAN_ID), 
                                         data = data, family = binomial)

summary(final_mixed_model)

### Model evaluation

#### R² and AIC

In [ ]:
r2(initial_mixed_model)
AIC(initial_mixed_model)

r2(final_mixed_model)
AIC(final_mixed_model)

#### Predictive Efficiency

In [ ]:
#Predicting based on final_mixed_model
mixed_model_predictions <- predict(final_mixed_model, newdata = data, type = "response")
#Convert probabilities to binary outcomes (i.e., if probability > 0.5, predict 1, else 0)
mixed_model_predicted_class <- ifelse(mixed_model_predictions > 0.5, "zwoelf", "dezember")
#Compare predicted values to actual values
mixed_model_accuracy <- mean(mixed_model_predicted_class == data$CURRENT)
mixed_model_accuracy

In [ ]:
#Calculate baseline accuracy, i.e., a dumb intercept-only model only ever predicting the most frequent outcome
counts <- table(data$CURRENT) 
dumb_model_accuracy <- max(counts) / sum(counts)
dumb_model_accuracy

In [ ]:
#McNemar's Test for significance against baseline
baseline_predicted_class <- rep(names(which.max(counts)), length(data$CURRENT))  #Create baseline predictions (always predict the most frequent outcome)
mcnemar_table <- table(
  model_correct = (mixed_model_predicted_class == data$CURRENT),
  baseline_correct = (baseline_predicted_class == data$CURRENT)) #Create a contingency table: Compare model and baseline predictions against actual values
mcnemar_result <- mcnemar.test(mcnemar_table) #Perform McNemar's Test
mcnemar_result

In [ ]:
#McNemar's Test for significance against fixed effects model
baseline_predicted_class <- rep(names(which.max(counts)), length(data$CURRENT))  #Create baseline predictions (always predict the most frequent outcome)
mcnemar_table <- table(
  mixed_model_correct = (mixed_model_predicted_class == data$CURRENT),
  fixed_model_correct = (fixed_model_predicted_class == data$CURRENT)) #Create a contingency table: Compare model and baseline predictions against actual values
mcnemar_result <- mcnemar.test(mcnemar_table) #Perform McNemar's Test
mcnemar_result

#### Multicollinearity / Pairwise Correlations

In [ ]:
library(car) #only importing it now as the module otherwise interferes with code execution above
vif(final_mixed_model)

#### Random Effect Structure

Given the substantially improved fit and AIC compared to the simple model, checking whether there are enough data points per group of HUMAN_ID, as too low a number could lead to overfitting.

In [ ]:
table(data$HUMAN_ID) 
mean(table(data$HUMAN_ID)) #more than 20 per group which is recommended
sd(table(data$HUMAN_ID))
length(unique(data$HUMAN_ID)) #slightly below rule of thumb of 30 groups

#### Leave-One-Out Cross Validation

Fitting models with one data point left out at each time, then having the models predict the left-out data point. Finally, seeing how well the models predict unseen data, giving an estimate of model performance. 

Note that this code takes a long time to fully execute.

In [ ]:
predictions <- numeric(nrow(data)) #vector for storing predictions

#Iterating over dataset
for (i in 1:nrow(data)) {
    print(i)
    
    train_data <- data[-i, ] #creating 'train_data' which leaves out the current data point

    #fitting model on 'train_data'
    model_loo <- glmer(CURRENT ~ PREVIOUS_SPEAKER_COMBINED * QUASI_PERSISTENCE +
                       PREVIOUS_BETA_ZWOELF + 
                       (1 | HUMAN_ID), 
                       data = train_data, family = binomial)

    #checking if model converged, if yes (messages are NULL)...
    if (is.null(model_loo@optinfo$conv$lme4$messages)) {
        #having 'model_loo' predict the current data point and storing it in 'predictions'
        predictions[i] <- predict(model_loo, newdata = data[i, , drop = FALSE], type = "response")
    } else { 
        predictions[i] <- NA #if not, storing NA (otherwise R fills 'predictions' for missing indices with 0 which, however, could also be a probability)
        }
}

Checking in how many cases no prediction could be made as the model did not converge on the remaining data points, filtering for valid predictions only.

In [ ]:
sum(is.na(predictions))
filtered_predictions <- predictions[!is.na(predictions)] 

Extracting the ground truth and filtering for values where a prediction could be made as a model was successfully fitted on the remaining data points. 

In [ ]:
actuals <- data$CURRENT
filtered_actuals <- actuals[!is.na(predictions)] 
length(filtered_actuals) == length(filtered_predictions)

Replacing probabilities in `filtered_predictions` such that values above 0.5 mean "zwoelf" was predicted ("zwoelf" is the modelled level) and values below mean "dezember" was predicted instead.

In [ ]:
predicted_class <- ifelse(filtered_predictions > 0.5, "zwoelf", "dezember")
predicted_class

Comparing ground truth with predicted class for each data point and calculating mean.

In [ ]:
mean(filtered_actuals == predicted_class)

### Visualisation

#### Coefficient Plot

In [ ]:
model_summary <- tidy(final_mixed_model, conf.int = TRUE) %>% filter(effect == "fixed")  #Include only fixed effects

plot <- ggplot(model_summary, aes(x = estimate, y = term, color = p.value < 0.05)) +
          geom_point(size = 3) +
          geom_errorbarh(aes(xmin = conf.low, xmax = conf.high), height = 0.2) +  
          geom_vline(xintercept = 0, linetype = "dashed") +
          labs(
            x = "Estimated Coefficient",
            y = "Predictors",
          ) +
          theme_minimal() +
          scale_color_manual(values = c("TRUE" = "black", "FALSE" = "gray")) +  #Grey out non-significant predictors
          coord_fixed(ratio = 0.5)

plot

#### Prediction Plot

Plot visualises the probability of observing "zwoelf" in CURRENT given different combinations of variants in PREVIOUS and SPEAKERS.

In [ ]:
#for "zwölf" in CURRENT
plot <- plot_model(final_mixed_model, type = "pred", 
                   terms = c("PREVIOUS_SPEAKER_COMBINED", "PREVIOUS_BETA_ZWOELF"), 
                   group = "HUMAN_ID", dpi = 300)

plot <- plot +
    aes(color = as.factor(group), shape = as.factor(group)) +
    scale_shape_manual(values = c(16, 17)) +
    guides(color = guide_legend(title = "PREVIOUS_BETA_ZWOELF"),
    shape = guide_legend(title = "PREVIOUS_BETA_ZWOELF")) +
    theme_minimal(base_size = 12) +
    theme(plot.title = element_blank(),
          legend.position = "right")

plot